Lorenzo Colletta, Giorgio Fantilli, Luca Lucioli

## Parte 1 - Descrizione del problema

Il dataset in esame include dati per la stima dei livelli di obesità negli individui provenienti dai paesi di Messico, Perù e Colombia, in base alle loro abitudini alimentari e alle condizioni fisiche. `TODO`

Di seguito vengono importate le librerie necessarie per scaricare i file, organizzare le strutture dati e disegnare i grafici.

In [2]:
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Caricamento dati e preprocessing

In [3]:
file_zip_url = "https://archive.ics.uci.edu/static/public/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition.zip"
file_zip_name = "obesity_data.zip"
file = "./ObesityDataSet_raw_and_data_sinthetic.csv"

if not os.path.exists(file_zip_name):
    from urllib.request import urlretrieve
    urlretrieve(file_zip_url, file_zip_name)
    from zipfile import ZipFile
    with ZipFile(file_zip_name) as f:
        f.extractall()

obesity_row_data = pd.read_csv(file)
obesity_row_data = obesity_row_data.rename(columns={"family_history_with_overweight" : "FHWO", "NObeyesdad" : "ObesityLevel"})

obesity_row_data.head(10)

,Gender,Age,Height,Weight,FHWO,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,ObesityLevel
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


Sopra viene mostrato un piccolo estratto del dataset in esame, dopo aver rinominato alcune feature per migliorarne la leggibilità e l'utilizzo.

Di seguito sono riportate le dimensioni in memoria, il numero di istanze non nulle e il tipo delle feature che compongono i dati raccolti nel dataset.


In [4]:
obesity_row_data.info(memory_usage="deep");

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Gender        2111 non-null   object 
 1   Age           2111 non-null   float64
 2   Height        2111 non-null   float64
 3   Weight        2111 non-null   float64
 4   FHWO          2111 non-null   object 
 5   FAVC          2111 non-null   object 
 6   FCVC          2111 non-null   float64
 7   NCP           2111 non-null   float64
 8   CAEC          2111 non-null   object 
 9   SMOKE         2111 non-null   object 
 10  CH2O          2111 non-null   float64
 11  SCC           2111 non-null   object 
 12  FAF           2111 non-null   float64
 13  TUE           2111 non-null   float64
 14  CALC          2111 non-null   object 
 15  MTRANS        2111 non-null   object 
 16  ObesityLevel  2111 non-null   object 
dtypes: float64(8), object(9)
memory usage: 1.1 MB


Osserviamo come molte delle feature sono di tipo `object`. Procediamo di seguito con la trasformazione in variabili categoriche e booleane per limitare la memoria occupata.

In [5]:
categorical = ["NCP", "CAEC", "CH2O", "FAF",
               "TUE", "CALC", "MTRANS", "ObesityLevel"];
boolean = ["FHWO", "FAVC", "SMOKE", "SCC"]

obesity_row_data[categorical] = obesity_row_data[categorical].astype("category")

obesity_row_data[boolean] = obesity_row_data[boolean].apply(lambda x: x.map({"yes": True, "no": False})).astype(bool)

obesity_row_data.info(verbose=False, memory_usage="deep");

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Columns: 17 entries, Gender to ObesityLevel
dtypes: bool(4), category(8), float64(4), object(1)
memory usage: 358.4 KB


Osserviamo come tutte le feature sono rileventi per il problema, e soprattutto che nessuna delle stesse presenta valori nulli. Non risulta perciò necessaria nessuna pulizia dei dati.

## Descrizione delle feature
Il dataset contiene le seguenti feature:
*   `Gender`: genere
*   `Age`: età
*   `Height`: altezza
*   `Weight`: peso
*   `FHWO`: indica se il soggetto ha familiari che sono o sono stati sovrappeso
*   `FAVC`: indica se il soggetto mangia spesso cibo ad alto contenuto calorico o meno
*   `FCVC`: indica se solitamente il soggetto mangia verdura durante i pasti o meno
*   `NCP`: indica quanti pasti consuma il soggetto durante la giornata (tra 1 e 2, tre, più di tre)
*   `CAEC`: indica se il soggetto mangia tra i pasti (no, qualche volta, frequentemente, sempre)
*   `SMOKE`: indica se il soggetto fuma o meno
*   `CH2O`: indica quanta acqua beve il soggetto durante il giorno (meno di un litro, tra uno e due litri, più di due litri)
*   `SCC`: indica se il soggetto monitora le calorie che assume durante il giorno
*   `FAF`: indica quanto spesso il soggetto svolge attività fisica durante la settimana (mai, da 1 a 2 volte a settimana, da 2 a 4 volte a settimana, da 4 a 5 volte a settimana)
*   `TUE`: indica quanto spesso il soggetto utilizza apparecchi tecnologici durante il giorno come telefono, videogiochi, tv, computer o altri (da 0 a 2 ore, da 3 a 5 ore, più di 5 ore)
*   `CALC`: indica quanto spesso il soggetto beve (mai, qualche volta, frequentemente, sempre)
*   `MTRANS`: indica quale mezzo di trasporto utilizza solitamente il soggetto (auto, moto, bicicletta, trasporto pubblico, a piedi)

La variabile target è `ObesityLevel`, che rappresenta il livello di obesità nelle seguenti classi: Insufficient Weight, Normal Weight, Overweight Level I, Overweight Level II, Obesity Type I, Obesity Type II, and Obesity Type III.

In [ ]:
obesity_row_data["ObesityLevel"].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)

Osserviamo dal grafico a torta che la variabile target `ObesityLevel` risulta essere bilanciata, ovvero il numero di istanze per le possibili classi sono pressochè identiche.  Il problema in esame risulta quindi essere bilanciato e non e' necessario utilizzare tecniche di bilanciamento delle classi.

In [ ]:
obesity_row_data.describe()

dasdasdadsadadsaddadadasdsadas sdsadadasd